<a href="https://colab.research.google.com/github/gserontHEVinci/BINV3220-IA/blob/main/BINV3220_mini_RAG_%C3%A9tudiants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Dans cette fiche, nous allons développer un système **RAG** (*Retrieval Augmented Generation*).  
L’objectif est de comprendre les différentes étapes nécessaires pour enrichir les capacités d’un modèle de langage en lui donnant accès à une base de connaissances externe.

Nous allons procéder **progressivement** :

1. **Tester le modèle d’embedding** :  
   Nous commencerons par construire des vecteurs de représentation (*embeddings*) à partir de quelques phrases simples, afin de vérifier que le modèle d’embedding fonctionne correctement.

2. **Stockage et recherche vectorielle** :  
   Nous utiliserons ensuite une base de données vectorielle pour stocker ces embeddings. Cela nous permettra de retrouver les phrases les plus proches d’une requête donnée grâce à une recherche par similarité.

3. **Mise en place du système RAG complet** :  
   Enfin, nous combinerons ces éléments pour construire un système qui :  
   - enrichit un *prompt* utilisateur avec les passages les plus pertinents retrouvés,  
   - envoie ce *prompt enrichi* à un modèle de langage,  
   - obtient une réponse générée à partir de ce contexte.

À la fin de cette fiche, vous aurez donc réalisé un prototype de RAG en plusieurs étapes, en comprenant le rôle de chaque composant : **embedding → recherche vectorielle → génération augmentée**.


## Installation, import et chargement du modèle d'embedding

Dans cette première cellule, nous allons instancier le modèle d’embedding.  
Ce modèle est disponible sur le repository Hugging Face à l’adresse suivante :  
[https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).  

Nous utiliserons le modèle **all-MiniLM-L6-v2**.  
Allez sur le lien indiqué, consultez la documentation, installez la librairie *Sentence Transformers*, puis instanciez ce modèle.


## Test de l'embedding

Dans cette cellule, à partir des phrases stockées dans la variable `sentences`, complétez le code pour calculer les embeddings de chacune d’entre elles, puis affichez les résultats.


In [ ]:
# Quelques phrases pour tester
sentences = [
    "L'intelligence artificielle transforme l'éducation.",
    "Les chats adorent dormir au soleil.",
    "Python est un langage de programmation populaire."
]


Phrase: L'intelligence artificielle transforme l'éducation.
Embedding (premiers 5 coefficients): [-0.00492159  0.08406187  0.01818705  0.01207972 -0.0274413 ]

Phrase: Les chats adorent dormir au soleil.
Embedding (premiers 5 coefficients): [-0.06642637 -0.00122354  0.03399962 -0.00836293  0.02679008]

Phrase: Python est un langage de programmation populaire.
Embedding (premiers 5 coefficients): [-0.03303327  0.02008152 -0.0155138  -0.06426464 -0.04411807]



## Introduction à la base de données vectorielle

Une **base de données vectorielle** est un outil qui permet de stocker et de rechercher des *vecteurs* — c’est-à-dire des représentations numériques d’objets comme des phrases, des images ou des documents.  
Chaque texte que nous traitons est transformé en un vecteur (*embedding*) qui capture son sens.  
En comparant ces vecteurs entre eux, on peut retrouver rapidement les textes les plus proches d’une question donnée.

Dans cette fiche, nous allons utiliser **FAISS** ([Facebook AI Similarity Search ](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)), une bibliothèque optimisée pour ce type de recherche.  
FAISS permet de stocker nos embeddings et de retrouver les plus proches en fonction d’une requête.


## Import de la base de donnée vectorielle

In [ ]:
!pip install -q faiss-cpu

import faiss



## Ajout et recherche d'embeddings dans la BD vectorielle

Objectif : construire un mini-index vectoriel avec **FAISS** et réaliser une **recherche par similarité cosinus**.

Regardez les exemples de code dans [doc FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/).

Nous allons repartir des embeddingsins calculés précédement.

* Créez un index FAISS de type `IndexFlatIP`.
* Ajoutez vos vecteurs à l’index.
* Vérifiez que `index.ntotal` correspond au nombre de phrases insérées.
* A partir de la requête en langage naturel (`query`), calculez son embedding, puis interrogez l’index pour obtenir les **k=2** passages les plus proches.
* Affichez la requête, puis les phrases retrouvées avec leur score.

In [ ]:
# Hypothèses :
# - Vous avez déjà importé FAISS : `import faiss`
# - Un modèle d'embedding est disponible dans la variable `model`
# - Vous avez déjà calculé les embeddings pour les phrases exemple

# TODO: Normalisation L2 des embeddings (cosine via Inner Product)
#    - Cette étape est cruciale pour simuler la similarité cosinus




# Création de l'index FAISS (Inner Product)
#
#  TODO: ajoutez-y les embeddings

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)



# TODO: Encodez-la phrase de query, normalisez-la, puis lancez la recherche top-k

query = [
    "Quels animaux aiment se reposer au soleil ?"
]


# Affichage des résultats
#    - Montrez la requête
#    - Pour chaque (indice, score) retourné, affichez la phrase correspondante



## Chargement d’un fichier PDF

Dans cette cellule, deux fonctions sont fournies : elles permettent de charger un fichier texte ou un fichier PDF et de renvoyer une chaîne de caractères contenant le texte extrait.


In [ ]:
from google.colab import files
import pathlib

# Étape 1 : upload du fichier
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # premier fichier uploadé
ext = pathlib.Path(file_path).suffix.lower()

# Étape 2 : fonctions de lecture
def read_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def read_pdf(path: str) -> str:
    from PyPDF2 import PdfReader
    reader = PdfReader(path)
    pages_text = []
    for p in reader.pages:
        t = p.extract_text() or ""
        pages_text.append(t)
    return "\n".join(pages_text)

# Étape 3 : extraction du texte
if ext == ".txt":
    full_text = read_txt(file_path)
elif ext == ".pdf":
    !pip install -q PyPDF2
    full_text = read_pdf(file_path)
else:
    raise ValueError("Format non supporté. Utilisez un .txt ou un .pdf")

print(f"Longueur du texte : {len(full_text)} caractères")
print("Aperçu (500 premiers caractères) :\n")
print(full_text[:500])


Saving Rapport final Comité expert IA programmes 20250115.pdf to Rapport final Comité expert IA programmes 20250115 (1).pdf
Longueur du texte : 78115 caractères
Aperçu (500 premiers caractères) :

 
 
 
 
 
 
 
 
 
 
       
 
      
   
Comité expert  
15 janvier 2025  
Adapter les programmes d’études 
pour tenir compte de  l’IA  
à l’Université de Sherbrooke  
Recomm andations  et invitation à l’action  
 
 
Rapport  du comité expert  | page 1   
 
Composition du Comité  
 
 Professeur  Dany  Baillargeon , Faculté des lettres et sciences humaines   
 Professeur  Daniel  Chamberland- Tremblay , École de gestion   
 Professeur  Jean -François  Desbiens, vice-doyen, Faculté des sciences


## Chunking

Lorsqu’on travaille avec de longs documents, il n’est pas possible de calculer directement un embedding unique pour l’ensemble du texte : cela dépasserait les limites des modèles et produirait des vecteurs trop généraux.  
La solution est de **découper le document en morceaux plus petits (chunks)**, chaque morceau recevant son propre embedding.  

Il existe de nombreuses stratégies de découpage (par phrases, par paragraphes, avec chevauchement, etc.).  
Dans cet exercice, nous allons utiliser une approche **naïve** : découper le texte en blocs de taille fixe, éventuellement avec un petit chevauchement pour ne pas couper trop brutalement les idées.  

**TODO** Implémentez la fonction décrite dans la cellule suivante pour réaliser ce chunking simple.
Testez-là

In [ ]:
# Exemple de document (hard-coded)
document_text = (
    "L’intelligence artificielle (IA) désigne un ensemble de méthodes permettant à des systèmes "
    "d’exécuter des tâches qui requièrent habituellement l’intelligence humaine : perception, "
    "raisonnement, apprentissage, langage. Dans les pipelines RAG, on découpe les documents en "
    "fragments (chunks) de taille contrôlée afin d’optimiser le rappel et la précision lors de la "
    "recherche vectorielle. Un chevauchement (overlap) entre chunks limite les coupures brutales "
    "de contexte et améliore la pertinence des passages récupérés."
)

def chunk_text(text: str, chunk_size: int = 300, overlap: int = 50):
    """
    Découpe naïve par caractères avec chevauchement.
    - chunk_size : taille cible d’un chunk (en caractères)
    - overlap    : nombre de caractères de recouvrement entre deux chunks consécutifs
    """


## Test du chunking

Testez d'abord sur le court texte proposé. Ensuite, chargez un document de votre choix, et observez la découpe de celui-ci. Qu'en pensez-vous ?



In [1]:
# Test du chunking

# Paramètres de découpage
chunk_size = 500
overlap = 50

## Pause réflexive sur le chunking

### 1. Problème du découpage naïf

* Quels sont les risques si l’on découpe simplement un document en blocs de longueur fixe (par ex. 500 caractères) ?
* Donnez un exemple de perte de sens ou de rupture logique que cela pourrait causer.

Voici un liens vers des techniques de chunking plus évoluées: [https://www.pinecone.io/learn/chunking-strategies](https://www.pinecone.io/learn/chunking-strategies) lisez cet article et répondez aux questions suivantes.

### 2. Méthodes proposées

L’article décrit plusieurs stratégies de chunking.
Pour chacune, résumez en **2–3 phrases** l’idée principale :

* **Fixed-size chunking** (découpage naïf).
* **Overlap chunking** (chevauchement).
* **Semantic chunking** (découpage selon le sens).
* **Recursive chunking** (utilisation de séparateurs hiérarchiques, comme paragraphes, phrases, mots).



### 3. Comparaison

* Quelle est selon vous la **principale limite** de la méthode *overlap chunking* ?
* En quoi le *semantic chunking* peut-il produire de meilleurs résultats que les autres méthodes ?
* Quel inconvénient peut poser ce type de méthode plus avancée ?


## Indexation du document

Nous allons tout de même durant cette fiche pousuivre avec le chunking "naïf".

À partir de ce que vous avez construit dans les cellules précédentes, découpez le document en *chunks*, calculez leurs embeddings et ajoutez-les dans la base de données vectorielle.


In [ ]:
# Indexation du document et peuplement de la base de données




Chunks: 174 | dim=384 | index.ntotal=174



## Construction d’un *prompt enrichi*

Jusqu’ici, nous avons appris à découper un document en *chunks*, à calculer leurs embeddings et à les stocker dans une base vectorielle.  
L’étape suivante est de construire un **prompt enrichi** : c’est-à-dire de prendre la question posée par l’utilisateur, de rechercher dans la base les passages les plus proches, puis de **concaténer ces extraits pertinents** avec la question.  
Ce prompt enrichi sera ensuite envoyé au modèle de langage, qui pourra répondre en s’appuyant sur le contenu du document plutôt que seulement sur sa mémoire interne.

La fonction `build_rag_prompt` que vous allez implémenter doit donc :
1. Prendre en entrée une question utilisateur (`query`).  
2. Rechercher les *k* chunks les plus pertinents dans l’index FAISS.  
3. Filtrer éventuellement selon un score minimal (`min_score`).  
4. Limiter la taille totale du contexte (`max_context_chars`).  
5. Construire un texte final contenant :  
   - un en-tête de contexte avec les extraits retenus,  
   - un en-tête pour la question utilisateur.  


Pour une question :

```text
Quels animaux aiment se reposer au soleil ?
```

Et une base de données vectorielle qui contient les embeddings des phrases suivantes :

* « Les chats aiment dormir au soleil. »
* « Les chiens sont de loyaux compagnons. »
* « L’intelligence artificielle transforme l’éducation. »
* « Python est un langage de programmation populaire. »

Le *prompt enrichi* pourrait ressembler à ceci :

```text
Contexte (extraits pertinents) :
Les chats aiment dormir au soleil.

Question :
Quels animaux aiment se reposer au soleil ?

Instruction: Réponds de manière factuelle et concise, en t'appuyant uniquement sur le contexte.
```

Vous allez maintenant compléter l’implémentation de la fonction `build_rag_prompt` afin de générer ce type de structure.



In [ ]:

# Cette cellule suppose que les variables suivantes existent déjà (depuis la 1ère cellule) :
# - full_text (texte extrait)
# - chunks (liste de morceaux de texte)
# - model (SentenceTransformer chargé)
# - embeddings (np.ndarray float32, normalisé L2)
# - index (faiss.IndexFlatIP construit et rempli)

import faiss
from typing import List, Tuple

def build_rag_prompt(
    query: str,
    k: int = 3,
    min_score: float = 0.0,
    max_context_chars: int = 2000,
    context_header: str = "Contexte (extraits pertinents) :",
    question_header: str = "Question :",
    separator: str = "\n---\n",
) -> Tuple[str, List[Tuple[str, float, int]]]:
    """
    Recherche les k chunks les plus proches de la requête et construit un prompt.
    Paramètres:
      - query : question utilisateur
      - k : nombre de chunks à insérer
      - min_score : seuil de similarité (cosine simulée via inner product après normalisation)
      - max_context_chars : limite de longueur totale pour le contexte concaténé
      - context_header, question_header : en-têtes
      - separator : séparateur entre extraits
    Retourne:
      - prompt (str)
      - hits : liste (chunk_text, score, chunk_index)
    """


## Test de l'enrichissement

Avec un pdf ou un document texte (pas trop court) de votre choix, poser une question pour générer un prompts enrichi. Injecter celui-ci dans votre chatbot favori (à la main) pour voir la réponse.

# Calcul de la similarité

Pour bien comprendre ce que représente la notion de similarité, veuillez prendre connaissance du contenu du colab [suivant](https://colab.research.google.com/drive/1G6FeB5Q1uaM5AUaxNa1zKIs-kTIqkRdC?usp=sharing).

Dans celui-ci vous trouverez un explication de différentes distance possibles et quels sont leur avantages relatifs.



## Réécriture du RAG sans FAISS

Jusqu’à présent, nous avons utilisé **FAISS** pour rechercher les passages les plus proches d’une requête dans la base vectorielle.  
Pour cette dernière étape, vous allez recréer cette partie du pipeline **sans utiliser FAISS**.

Vous devez écrire votre propre fonction qui compare l’embedding d’une requête avec ceux de la base, calcule une mesure de similarité ou de distance, et retourne les résultats les plus proches.  

Utilisez la mesure de proximité la plus pertinente.

**Attendu :**  
- une fonction claire et autonome,  
- un appel qui retourne les *k* passages les plus proches pour une requête donnée,  
- un affichage qui montre les phrases retenues et leur score de similarité.

Vous ne devez pas refaire l'enrichissement du prompt.

Le but de l’exercice est de comprendre ce que fait FAISS « sous le capot », en implémentant vous-même la mécanique de base.


In [ ]:
# Mini retrieval